In [1]:
!pip install ipython-autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%load_ext autotime

time: 183 µs (started: 2022-07-04 15:33:51 +00:00)


In [3]:
import glob
import pandas as pd
import nltk
nltk.download('popular');
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
from collections import Counter
import numpy as np
from collections import OrderedDict

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

time: 10.6 s (started: 2022-07-04 15:33:54 +00:00)


[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection popular


In [4]:
import re

time: 1.01 ms (started: 2022-07-04 15:34:10 +00:00)


In [34]:
data = pd.read_csv('combined_relevant_dataset.csv')
data.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


time: 230 ms (started: 2022-07-04 15:47:00 +00:00)


In [29]:
data.shape

(17562, 35)

time: 5.61 ms (started: 2022-07-04 15:45:06 +00:00)


In [12]:
data['Input Data'] = data['Name'].apply(str) + " " + data['Genres'].apply(str) + " " + data['Type'].apply(str) + " " + data['Aired'].apply(str) + " " + data['Studios'].apply(str) + " " + data['sypnopsis'].apply(str)
data.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Input Data
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"Cowboy Bebop Action, Adventure, Comedy, Drama,..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,"Cowboy Bebop: Tengoku no Tobira Action, Drama,..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0,"Trigun Action, Sci-Fi, Adventure, Comedy, Dram..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0,"Witch Hunter Robin Action, Mystery, Police, Su..."
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0,"Bouken Ou Beet Adventure, Fantasy, Shounen, Su..."


time: 42.1 ms (started: 2022-07-04 15:38:45 +00:00)


In [13]:
inp_data = data[['Input Data']]
inp_data.head()

,Input Data
0,"Cowboy Bebop Action, Adventure, Comedy, Drama,..."
1,"Cowboy Bebop: Tengoku no Tobira Action, Drama,..."
2,"Trigun Action, Sci-Fi, Adventure, Comedy, Dram..."
3,"Witch Hunter Robin Action, Mystery, Police, Su..."
4,"Bouken Ou Beet Adventure, Fantasy, Shounen, Su..."


time: 23.1 ms (started: 2022-07-04 15:38:55 +00:00)


In [36]:
names = data['Name']
names.head()

0                       Cowboy Bebop
1    Cowboy Bebop: Tengoku no Tobira
2                             Trigun
3                 Witch Hunter Robin
4                     Bouken Ou Beet
Name: Name, dtype: object

time: 16.7 ms (started: 2022-07-04 15:48:06 +00:00)


In [14]:
dic = {}
for i in range (10):
  dic[i] = str(data.loc[i, 'Input Data'])



time: 8.45 ms (started: 2022-07-04 15:39:11 +00:00)


In [15]:
dic

{0: 'Cowboy Bebop Action, Adventure, Comedy, Drama, Sci-Fi, Space Apr 3, 1998 to Apr 24, 1999 TV',
 1: 'Cowboy Bebop: Tengoku no Tobira Action, Drama, Mystery, Sci-Fi, Space Sep 1, 2001 Movie',
 2: 'Trigun Action, Sci-Fi, Adventure, Comedy, Drama, Shounen Apr 1, 1998 to Sep 30, 1998 TV',
 3: 'Witch Hunter Robin Action, Mystery, Police, Supernatural, Drama, Magic Jul 2, 2002 to Dec 24, 2002 TV',
 4: 'Bouken Ou Beet Adventure, Fantasy, Shounen, Supernatural Sep 30, 2004 to Sep 29, 2005 TV',
 5: 'Eyeshield 21 Action, Sports, Comedy, Shounen Apr 6, 2005 to Mar 19, 2008 TV',
 6: 'Hachimitsu to Clover Comedy, Drama, Josei, Romance, Slice of Life Apr 15, 2005 to Sep 27, 2005 TV',
 7: 'Hungry Heart: Wild Striker Slice of Life, Comedy, Sports, Shounen Sep 11, 2002 to Sep 10, 2003 TV',
 8: 'Initial D Fourth Stage Action, Cars, Sports, Drama, Seinen Apr 17, 2004 to Feb 18, 2006 TV',
 9: 'Monster Drama, Horror, Mystery, Police, Psychological, Seinen, Thriller Apr 7, 2004 to Sep 28, 2005 TV'}

time: 6.93 ms (started: 2022-07-04 15:39:16 +00:00)


In [16]:
def wordList_removePuncs(doc_dict):
    stop = stopwords.words('english') + list(string.punctuation) + ['\n']
    wordList = []
    for doc in doc_dict.values():
        for word in word_tokenize(doc.lower().strip()): 
            if not word in stop:
                wordList.append(word)
    return wordList

time: 4.41 ms (started: 2022-07-04 15:39:21 +00:00)


In [17]:
def give_path(fld_path):                             #give path of the folder containing all documents
    dic = {}
    file_names = glob.glob(fld_path)
    files_150 = file_names[0:10]
    for file in files_150:
        name = file.split('/')[-1]
        with open(file, 'r', errors='ignore') as f:
            data = f.read()
        dic[name] = data
    return dic

time: 7.92 ms (started: 2022-07-04 15:39:24 +00:00)


In [18]:
def termFrequencyInDoc(vocab, doc_dict):
    tf_docs = {}
    for doc_id in doc_dict.keys():
        tf_docs[doc_id] = {}
    
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_docs[doc_id][word] = doc.count(word)
    return tf_docs

time: 4.77 ms (started: 2022-07-04 15:39:27 +00:00)


In [19]:
def wordDocFre(vocab, doc_dict):
    df = {}
    for word in vocab:
        frq = 0
        for doc in doc_dict.values():
#             if word in doc.lower().split():
            if word.lower() in word_tokenize(doc.lower().strip()):
                frq = frq + 1
        df[word] = frq
    return df

time: 4.55 ms (started: 2022-07-04 15:39:30 +00:00)


In [22]:
def inverseDocFre(vocab,doc_fre,length):
    idf= {} 
    for word in vocab:     
        idf[word] = np.log2((length+1) / doc_fre[word])
    return idf

time: 2.77 ms (started: 2022-07-04 15:39:47 +00:00)


In [23]:
def tfidf(vocab,tf,idf_scr,doc_dict):
    tf_idf_scr = {}
    for doc_id in doc_dict.keys():
        tf_idf_scr[doc_id] = {}
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_idf_scr[doc_id][word] = tf[doc_id][word] * idf_scr[word]
    return tf_idf_scr

time: 9.12 ms (started: 2022-07-04 15:39:50 +00:00)


In [24]:
def vectorSpaceModel(query, doc_dict,tfidf_scr):
    out_list = []
    query_vocab = []
    for word in query.split():
        if word not in query_vocab:
            query_vocab.append(word)

    query_wc = {}
    for word in query_vocab:
        query_wc[word] = query.lower().split().count(word)
    
    relevance_scores = {}
    for doc_id in doc_dict.keys():
        score = 0
        for word in query_vocab:
            score += query_wc[word] * tfidf_scr[doc_id][word]
        relevance_scores[doc_id] = score
    sorted_value = OrderedDict(sorted(relevance_scores.items(), key=lambda x: x[1], reverse = True))
    top_5 = {k: sorted_value[k] for k in list(sorted_value)[:5]}
    return top_5

time: 21.6 ms (started: 2022-07-04 15:39:55 +00:00)


In [62]:
if __name__  == "__main__":
   #path = '/content/ACL txt/*.txt'
   out_dic = {}
   out_ls_1 = []
   out_ls_2 = []
   out_ls_3 = []
   docs = dic                        #returns a dictionary of all docs
   M = len(docs)                                 #number of files in dataset
   w_List = wordList_removePuncs(docs)           #returns a list of tokenized words
   vocab = list(set(w_List))                     #returns a list of unique words
   tf_dict = termFrequencyInDoc(vocab, docs)     #returns term frequency
   df_dict = wordDocFre(vocab, docs)             #returns document frequencies
   idf_dict = inverseDocFre(vocab,df_dict,M)     #returns idf scores
   tf_idf = tfidf(vocab,tf_dict,idf_dict,docs)   #returns tf-idf socres
   
   query1 = 'cowboy 2001'
   #query2 = 'adventure tv '
   query3 = '2004 tv'

   top1 = vectorSpaceModel(query1, docs,tf_idf)    #returns top 5 documents using VSM
   #top2 = vectorSpaceModel(query2, docs,tf_idf)    #returns top 5 documents using VSM
   top3 = vectorSpaceModel(query3, docs,tf_idf)    #returns top 5 documents using VSM

   #for key in top1:
    # out_ls_1.append(key)

   #for key in top2:
   #  out_ls_2.append(key)

   #for key in top3:
   #  out_ls_3.append(key)

   print('Top 5 Documents for Query 1: \n', top1)
   print('\n')
   #print('Top 5 Documents for Query 2: \n', top2)
   #print('\n')
   print('Top 5 Documents for Query 2: \n', top3)
   #print('\n')
   #print(out_ls_1)
  # print('\n')
  # print(out_ls_2)
  # print('\n')
   #print(out_ls_3)


Top 5 Documents for Query 1: 
 {1: 3.4594316186372973, 0: 0.0, 2: 0.0, 3: 0.0, 4: 0.0}


Top 5 Documents for Query 2: 
 {4: 1.874469117916141, 8: 1.874469117916141, 9: 1.874469117916141, 0: 0.0, 1: 0.0}
time: 145 ms (started: 2022-07-04 16:39:08 +00:00)


In [63]:
for key in top1:
  out_ls_1.append(key)

for key in top3:
  out_ls_3.append(key)

print(out_ls_1)
print(out_ls_3)

[1, 0, 2, 3, 4]
[4, 8, 9, 0, 1]
time: 5.9 ms (started: 2022-07-04 16:39:12 +00:00)


In [59]:
for key in top3:
  out_ls_3.append(key)

print(out_ls_3)

[4, 8, 9, 0, 1]
time: 3.73 ms (started: 2022-07-04 16:38:03 +00:00)


In [37]:
for i in range(len(out_ls_1)):
  k = out_ls_1[i]
  print(str(names.loc[k]))

Cowboy Bebop: Tengoku no Tobira
Cowboy Bebop
Trigun
Witch Hunter Robin
Bouken Ou Beet
time: 4.82 ms (started: 2022-07-04 15:48:19 +00:00)


In [64]:
for i in range(len(out_ls_3)):
  k = out_ls_3[i]
  print(str(names.loc[k]))

Bouken Ou Beet
Initial D Fourth Stage
Monster
Cowboy Bebop
Cowboy Bebop: Tengoku no Tobira
time: 6.69 ms (started: 2022-07-04 16:39:59 +00:00)
